### Developing a real stratergy to invest in the stock market

When developing our stratergy, the goal will be to acquire good and useful information about each stock. We then need to check which parameters play a role in explaining variation in the data and which simply contribute noise to the system. Once we have generated and calculated the financial indicators and then optimised the tuning of these params using something like simulated annealing to search the complex solution space we can test and compare our stratergy against the S&P500 real life data. 

Sentiment data could be useful, financial statements even better, unsure how to extract, useful information from these datasets.

Looking for monthly readjustments for our stock portfolio. Further investigation should be carried 
on arbitrage. 

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [3]:
tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2015-01-01"
end_date = "2023-12-31"

In [4]:
def download_stock_data(tickers, start_date, end_date):
    stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    return stock_data

data = download_stock_data(tickers, start_date, end_date)

# data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

# data = pd.read_csv("C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/AAPL_stock_data.csv")

data.head()

# data.to_csv('AAPL_stock_data.csv', index=True)

[*********************100%***********************]  3 of 3 completed


Ticker          GOOGL                                                        \
Price            Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2015-01-02  26.629999  26.790001  26.393999  26.477501  26.381865  26480000   
2015-01-05  26.357500  26.399500  25.887501  25.973000  25.879185  41182000   
2015-01-06  26.025000  26.060499  25.277500  25.332001  25.240503  54456000   
2015-01-07  25.547501  25.574499  25.182501  25.257500  25.166271  46918000   
2015-01-08  25.075500  25.375000  24.750999  25.345501  25.253954  73054000   

Ticker           MSFT                                                        \
Price            Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2015-01-02  46.660000  47.419998  46.540001  46.759998  40.232861  27913900   
2015-01-05  46.369999  46.730000  46.250000  46.330002  39.862885  39673900   
2015-01-06  46.380001  46.750000  45.540001  45.650002  39.277805  36447900   
2015-01-07  45.980000  46.459999  45.490002  46.230000  39.776833  29114100   
2015-01-08  46.750000  47.750000  46.720001  47.590000  40.946983  29645200   

Ticker           AAPL                                                         
Price            Open       High        Low      Close  Adj Close     Volume  
Date                                                                          
2015-01-02  27.847500  27.860001  26.837500  27.332500  24.347174  212818400  
2015-01-05  27.072500  27.162500  26.352501  26.562500  23.661272  257142000  
2015-01-06  26.635000  26.857500  26.157499  26.565001  23.663502  263188400  
2015-01-07  26.799999  27.049999  26.674999  26.937500  23.995316  160423600  
2015-01-08  27.307501  28.037500  27.174999  27.972500  24.917271  237458000

In [5]:
multi_index_data = (
    data.stack(level=0, future_stack=True)
    .swaplevel()
    .sort_index()
)

multi_index_data

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2015-01-02   27.847500   27.860001   26.837500   27.332500   24.347174   
       2015-01-05   27.072500   27.162500   26.352501   26.562500   23.661272   
       2015-01-06   26.635000   26.857500   26.157499   26.565001   23.663502   
       2015-01-07   26.799999   27.049999   26.674999   26.937500   23.995316   
       2015-01-08   27.307501   28.037500   27.174999   27.972500   24.917271   
...                       ...         ...         ...         ...         ...   
MSFT   2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543915   
       2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
       2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
       2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
       2023-12-29  376.000000  377.160004  373.480011  376.040009  373.996002   

Price                 Volume  
Ticker Date                   
AAPL   2015-01-02  212818400  
       2015-01-05  257142000  
       2015-01-06  263188400  
       2015-01-07  160423600  
       2015-01-08  237458000  
...                      ...  
MSFT   2023-12-22   17091100  
       2023-12-26   12673100  
       2023-12-27   14905400  
       2023-12-28   14327000  
       2023-12-29   18723000  

[6792 rows x 6 columns]

In [6]:
### Define and calcuate RSI 
def calculate_rsi(data, column='Adj Close', window=14):
    """
    Calculate the RSI for each stock in a multi-index df
    """
    def rsi_per_stock(data):
    
        if not isinstance(data, pd.DataFrame):
            raise ValueError("Input data must be a pandas DataFrame")
    
        if column not in data.columns:
            raise ValueError(f"Column '{column}' not found in the DataFrame")
    
        # Calculate daily price changes (delta)
        delta = data['Adj Close'].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.rolling(window=window, min_periods=window).mean()
        avg_loss = losses.rolling(window=window, min_periods=window).mean()
    
        rs = avg_gain/avg_loss
        data['RSI'] = 100 - (100/ (1+rs)) # Ratio of avg gains/ avg losses during the window -> normalised to be 0 < rsi < 1
        
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_per_stock)

def calculate_rsi_ewma(data, column="Adj Close", window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    def rsi_ewma_per_stock(data):
        # Calculate daily price changes (delta)
        delta = data[column].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.ewm(span=window, adjust=False).mean()
        avg_loss = losses.ewm(span=window, adjust=False).mean()
    
        rs = avg_gain/avg_loss
        rsi = 100 - (100/ (1+rs))

        data['RSI_EWMA'] = 100 - (100/ (1+rs)) # Ratio of exponentially smoothed gains/ losses during the window -> normalised to be 0 < rsi < 1
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_ewma_per_stock)


def calculate_bollinger_bands(data, window=20):
    """
    Calculates Bollinger bands for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")
    
    def calculate_bb_per_stock(data):
        bb_data = data['Adj Close']

        rolling_mean = bb_data.rolling(window=window).mean()
        rolling_std = bb_data.rolling(window=window).std()

        data[('Bollinger', 'Middle Band')] = rolling_mean
        data[('Bollinger', 'Upper Band')] = rolling_mean + (2 * rolling_std)
        data[('Bollinger', 'Lower Band')] = rolling_mean - (2 * rolling_std)

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_bb_per_stock)
    

In [7]:
column = "Adj Close"

df = calculate_rsi(multi_index_data, column)

df = calculate_rsi_ewma(df, column=column, window=14)

df = calculate_bollinger_bands(df)

df.head(25)

Price                   Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
AAPL   2015-01-02  27.847500  27.860001  26.837500  27.332500  24.347174   
       2015-01-05  27.072500  27.162500  26.352501  26.562500  23.661272   
       2015-01-06  26.635000  26.857500  26.157499  26.565001  23.663502   
       2015-01-07  26.799999  27.049999  26.674999  26.937500  23.995316   
       2015-01-08  27.307501  28.037500  27.174999  27.972500  24.917271   
       2015-01-09  28.167500  28.312500  27.552500  28.002501  24.943993   
       2015-01-12  28.150000  28.157499  27.200001  27.312500  24.329357   
       2015-01-13  27.857500  28.200001  27.227501  27.555000  24.545372   
       2015-01-14  27.260000  27.622499  27.125000  27.450001  24.451836   
       2015-01-15  27.500000  27.514999  26.665001  26.705000  23.788204   
       2015-01-16  26.757500  26.895000  26.299999  26.497499  23.603369   
       2015-01-20  26.959999  27.242500  26.625000  27.180000  24.211321   
       2015-01-21  27.237499  27.764999  27.067499  27.387501  24.396164   
       2015-01-22  27.565001  28.117500  27.430000  28.100000  25.030844   
       2015-01-23  28.075001  28.437500  27.882500  28.245001  25.160004   
       2015-01-26  28.434999  28.590000  28.200001  28.275000  25.186722   
       2015-01-27  28.105000  28.120001  27.257500  27.285000  24.304859   
       2015-01-28  29.407499  29.530001  28.827499  28.827499  25.678885   
       2015-01-29  29.080000  29.797501  28.889999  29.725000  26.478365   
       2015-01-30  29.600000  30.000000  29.212500  29.290001  26.090872   
       2015-02-02  29.512501  29.792500  29.020000  29.657499  26.418228   
       2015-02-03  29.625000  29.772499  29.402500  29.662500  26.422684   
       2015-02-04  29.625000  30.127501  29.577499  29.889999  26.625334   
       2015-02-05  30.004999  30.057501  29.812500  29.985001  26.815371   
       2015-02-06  30.004999  30.062500  29.612499  29.732500  26.589560   

Price                 Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                    
AAPL   2015-01-02  212818400        NaN        NaN                       NaN   
       2015-01-05  257142000        NaN   0.000000                       NaN   
       2015-01-06  263188400        NaN   0.049986                       NaN   
       2015-01-07  160423600        NaN   7.950770                       NaN   
       2015-01-08  237458000        NaN  26.561874                       NaN   
       2015-01-09  214798000        NaN  27.055108                       NaN   
       2015-01-12  198603200        NaN  22.962117                       NaN   
       2015-01-13  268367600        NaN  27.415107                       NaN   
       2015-01-14  195826400        NaN  26.645591                       NaN   
       2015-01-15  240056000        NaN  21.666867                       NaN   
       2015-01-16  314053200        NaN  20.439514                       NaN   
       2015-01-20  199599600        NaN  34.517236                       NaN   
       2015-01-21  194303600        NaN  38.344501                       NaN   
       2015-01-22  215185600        NaN  49.937015                       NaN   
       2015-01-23  185859200  57.671231  52.053820                       NaN   
       2015-01-26  222460000  66.442547  52.532873                       NaN   
       2015-01-27  382274800  55.811123  38.053092                       NaN   
       2015-01-28  585908400  62.830962  58.578707                       NaN   
       2015-01-29  337745600  62.123880  66.116239                       NaN   
       2015-01-30  334982000  58.434339  60.009242                 24.639235   
       2015-02-02  250956400  66.039641  63.312499                 24.742788   
       2015-02-03  207662800  64.899229  63.360029                 24.880858   
       2015-02-04  28059880

In [8]:
def calculate_macd(data, column="Adj Close", fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD, Signal Line, and MACD Histogram for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_macd_per_stock(data):
        macd_data = data['Adj Close']
        
        fast_ema = macd_data.ewm(span=fast_period, adjust=False).mean()
        slow_ema = macd_data.ewm(span=slow_period, adjust=False).mean()
        
        data['MACD Line'] = macd_line = fast_ema - slow_ema
        data['Signal Line'] = signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()

        # Calculate MACD histogram 
        data['Histogram'] = macd_line - signal_line

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_macd_per_stock)


def calculate_atr(data, window=14):
    """
    Calculate the average true range for multi_index_df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_atr_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Adj Close']

        # Calculate True Range (TR)
        true_range = pd.DataFrame({
            'high_low': high - low,
            'high_prev_close': (high - close.shift()).abs(),
            'low_prev_close': (low - close.shift()).abs()
        }).max(axis=1)

        # Calculate ATR using EMA
        data['ATR'] = true_range.ewm(span=window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_atr_per_stock)

def calculate_roc(data, column="Adj Close", periods=14):
    """
    Calculate Rate of Change (RoC) for multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_roc_per_stock(data):
        # Extract relevant column
        roc_data = data[column]
        
        # Calculate RoC
        data['RoC'] = ((roc_data - roc_data.shift(periods)) / roc_data.shift(periods)) * 100

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_roc_per_stock)


def calculate_obv(data):
    """
    Calculate On-Balance Volume (OBV) for a multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_obv_per_stock(data):
        # Extract relevant columns
        close = data['Close']
        volume = data['Volume']

        # Calculate OBV
        obv = (volume * np.sign(close.diff())).fillna(0).cumsum()

        # Add OBV to the DataFrame
        data['OBV'] = obv

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_obv_per_stock)

def calculate_moving_averages(data, column="Adj Close", sma_window=50, ema_window=50):
    """
    Calculate SMA and EMA for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_averages_per_stock(data):
        # Extract relevant column
        price_data = data[column]
        
        # Calculate SMA
        data[f"SMA_{sma_window}"] = price_data.rolling(window=sma_window).mean()
        
        # Calculate EMA
        data[f"EMA_{ema_window}"] = price_data.ewm(span=ema_window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_averages_per_stock)

def calculate_garman_klass_volatility(data):
    """
    Calculate Garman-Klass Volatility for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_gk_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Close']
        open_price = data['Open']

        # Calculate Garman-Klass Volatility
        gk_volatility = (
            0.5 * (np.log(high / low) ** 2)
            - (2 * np.log(2) - 1) * (np.log(close / open_price) ** 2)
        )

        # Add Garman-Klass Volatility to the DataFrame
        data['GK Volatility'] = np.sqrt(gk_volatility)

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_gk_per_stock)

In [9]:
df = calculate_macd(df)

df = calculate_atr(df)

df = calculate_roc(df)

df = calculate_obv(df)

df = calculate_moving_averages(df)

df = calculate_garman_klass_volatility(df)

df.tail()

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
MSFT   2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543915   
       2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
       2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
       2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
       2023-12-29  376.000000  377.160004  373.480011  376.040009  373.996002   

Price                Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                   
MSFT   2023-12-22  17091100  56.647090  56.133461                371.498645   
       2023-12-26  12673100  52.844259  56.326091                371.302220   
       2023-12-27  14905400  57.639910  54.297721                370.873065   
       2023-12-28  14327000  56.453010  57.886393                370.695534   
       2023-12-29  18723000  52.916548  60.154046                370.552815   

Price              (Bollinger, Upper Band)  (Bollinger, Lower Band)  \
Ticker Date                                                           
MSFT   2023-12-22               379.407019               363.590271   
       2023-12-26               378.869929               363.734511   
       2023-12-27               377.064265               364.681866   
       2023-12-28               376.352866               365.038201   
       2023-12-29               375.675097               365.430534   

Price              MACD Line  Signal Line  Histogram       ATR       RoC  \
Ticker Date                                                                
MSFT   2023-12-22   2.826479     3.868914  -1.042435  5.790795  1.473687   
       2023-12-26   2.787810     3.652693  -0.864883  5.604834  0.574471   
       2023-12-27   2.678935     3.457941  -0.779007  5.182388  1.428961   
       2023-12-28   2.659102     3.298174  -0.639072  5.081174  1.167263   
       2023-12-29   2.673559     3.173251  -0.499691  4.926336  0.483662   

Price                       OBV      SMA_50      EMA_50  GK Volatility  
Ticker Date                                                             
MSFT   2023-12-22  2.290790e+09  357.744221  359.437706       0.004425  
       2023-12-26  2.303463e+09  358.690923  359.954796       0.006458  
       2023-12-27  2.288558e+09  359.528422  360.428597       0.004208  
       2023-12-28  2.302885e+09  360.401502  360.931009       0.004331  
       2023-12-29  2.321608e+09  361.328409  361.443362       0.006933

In [10]:
def calculate_rolling_beta(data, market_column='MArket Pct Change', stock_column="Adj Close", window=30):
    """
    Calculate Rolling Beta for a MultiIndex DataFrame.

    The rolling Beta feature measures a stock's sensitivity to the market's movements over a rolling window
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_beta_per_stock(data):
        # Calculate daily stock and market returns
        stock_returns = data[stock_column].pct_change()
        market_returns = data[market_column]

        # Rolling covariance and variance
        rolling_cov = stock_returns.rolling(window=window).cov(market_returns)
        rolling_var = market_returns.rolling(window=window).var()

        # Calculate Rolling Beta
        data['Rolling Beta'] = rolling_cov / rolling_var

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_beta_per_stock)

In [13]:
# Define the market index (e.g., S&P 500)
market_ticker = "^GSPC"

# Fetch historical data
market_data = yf.download(market_ticker, start="2015-01-01", end="2023-12-31", progress=False)

market_data['Market Pct Change'] = market_data['Adj Close'].pct_change()

# Get unique tickers from stock data
unique_tickers = df.index.get_level_values('Ticker').unique()

# Expand the market data for each ticker

# Creates the multi-index for the df
market_data_expanded = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [unique_tickers, market_data.index], 
        names=['Ticker', 'Date']
    )
)

# Replicates the data for however many unique tickers there are
market_data_expanded['Market Pct Change'] = np.tile(
    market_data['Market Pct Change'].values, len(unique_tickers)
)

# Combines this data now with our stock data
df = df.merge(
    market_data_expanded,
    left_index=True,
    right_index=True,
    how='left'
)




# print(df.groupby(level='Ticker')['Market Pct Change'].head())
# df = df.drop(columns='Market Returns')

In [14]:
df = calculate_rolling_beta(df, market_column='Market Pct Change', stock_column='Adj Close')

### Looking at influential figures and politicians investments

In [2]:
import requests
from bs4 import BeautifulSoup

# url = "https://www.quiverquant.com/congresstrading/"

# response = requests.get(url)
# if response.status_code == 200:
#     print("Successfully fetched webpage content!")
#     html_content = response.content
# else:
#     print(f"Failed to fetch webpage. Status code: {response.status_code}")

#Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find('table')

if table:
    print("Table found!")
else:
    print("No table found")


Table found!


In [19]:
# Extract all rows from the table
rows = table.find_all('tr')

# Loop through rows and extract cell data
data = []
for row in rows[1:]:
    cells = row.find_all('td')
    row_data = [cell.text.strip() for cell in cells] # Get text from each cell without whitespace
    data.append(row_data)

# print("Extracted data:", data[:5])

cleaned_data = []
for row in data:
    # Clean each cell in the row by stripping newlines and excessive spaces
    cleaned_row = [cell.replace("\n", "").replace("  ", " ").strip() for cell in row]
    cleaned_row = [" ".join(cell.split()) for cell in row]
    cleaned_data.append(cleaned_row)

print("Cleaned Data:", cleaned_data[:5])

Cleaned Data: [['AY Atlantica Yield Plc Stock', 'Sale (Full) $1,001 - $15,000', 'Thomas R. Carper Senate / D', 'Jan. 02, 2025', 'Dec. 13, 2024', '-', '-'], ['- SKI MASK DOG CT', 'Purchase $1,001 - $15,000', 'Mike Collins House / R', 'Jan. 01, 2025', 'Dec. 03, 2024', '-', '-'], ['ABT Abbott Laboratories Common Stock Stock', 'Sale (Partial) $15,001 - $50,000', 'Markwayne Mullin Senate / R', 'Dec. 31, 2024', 'Dec. 11, 2024', '-', '3.16%'], ['AMAT Applied Materials, Inc. - Common Stock Stock', 'Purchase $15,001 - $50,000', 'Markwayne Mullin Senate / R', 'Dec. 31, 2024', 'Dec. 11, 2024', '-', '-0.47%'], ['AXON Axon Enterprise, Inc. - Common Stock Stock', 'Sale (Full) $15,001 - $50,000', 'Markwayne Mullin Senate / R', 'Dec. 31, 2024', 'Dec. 18, 2024', '-', '-1.90%']]


In [6]:
# Extract column names from the header row
header_row = rows[0]

headers = [th.text.strip() for th in header_row.find_all('th')]

print("Headers:", headers)

Headers: ['Stock', 'Transaction', 'Politician', 'Filed', 'Traded', 'Description', '']


In [40]:
import pandas as pd

# Refine our data cleaning to obtain singular elements
refined_data = []
for row in cleaned_data:
    # Split the Stock Column into Ticker and Company Name
    stock_parts = row[0].split(maxsplit=1)
    ticker = stock_parts[0].strip()
    company = stock_parts[1].strip() if len(stock_parts) > 1 else ""

    # Split the Transaction Column into Type and Amount Range
    transaction_parts = row[1].split("$", 1)
    transaction_type = transaction_parts[0].strip()
    # amount_range = "$" + transaction_parts[1].strip() if len(transaction_parts)>1 else ""
    if len(transaction_parts) > 1:
        # Further split the amount range to capture both numbers
        amount_range_parts = transaction_parts[1].split("-", 1)
        lower_amount = amount_range_parts[0].replace(",", "").strip()
        upper_amount = amount_range_parts[1].replace(",", "").strip() if len(amount_range_parts)>1 else ""
        upper_amount = upper_amount.replace("$", "").strip()
        amount_range = f"{lower_amount} - {upper_amount}"
    else:
        amount_range = ""
    
    # Split the Transaction Column into Type and Amount Range
    if "Senate" in row[2] or "House" in row[2]:
        split_point = row[2].find("Senate") if "Senate" in row[2] else row[2].find("House")
        politician_name = row[2][:split_point].strip()
        politician_affiliation = row[2][split_point:].strip()
    else:
        politician_name = row[2].strip()
        politican_affiliation = ""

    # Clean and parse the performance column
    performance = row[6].replace("%", "").strip()
    performance = float(performance) if performance not in ["-", ""] else None

    refined_data.append([
    ticker, company, transaction_type, amount_range, 
    politician_name, politician_affiliation, 
    row[3], row[4], performance
    ])

columns = [
    "Ticker", "Company", "Transaction Type", "Amount Range", 
    "Politician", "Affiliation", 
    "Reported Date", "Transaction Date", "Performance"
]

# Convert our data in a pandas df
politician_df = pd.DataFrame(refined_data, columns=columns)

# Convert dates to datetime format
politician_df["Reported Date"] = pd.to_datetime(df["Reported Date"], errors="coerce")
politician_df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


politician_df.head()

,Ticker,Company,Transaction Type,Amount Range,Politician,Affiliation,Reported Date,Transaction Date,Performance
0,AY,Atlantica Yield Plc Stock,Sale (Full),1001 - 15000,Thomas R. Carper,Senate / D,2025-01-02,2024-12-13,NaN
1,-,SKI MASK DOG CT,Purchase,1001 - 15000,Mike Collins,House / R,2025-01-01,2024-12-03,NaN
2,ABT,Abbott Laboratories Common Stock Stock,Sale (Partial),15001 - 50000,Markwayne Mullin,Senate / R,2024-12-31,2024-12-11,3.16
3,AMAT,"Applied Materials, Inc. - Common Stock Stock",Purchase,15001 - 50000,Markwayne Mullin,Senate / R,2024-12-31,2024-12-11,-0.47
4,AXON,"Axon Enterprise, Inc. - Common Stock Stock",Sale (Full),15001 - 50000,Markwayne Mullin,Senate / R,2024-12-31,2024-12-18,-1.90


In [42]:
politician_df.tail()

,Ticker,Company,Transaction Type,Amount Range,Politician,Affiliation,Reported Date,Transaction Date,Performance
145,TSCO,TRACTOR SUPPLY COMPANY - COMMON STOCK ST,Purchase,1001 - 15000,Jared Moskowitz,House / D,2024-12-09,2024-11-08,-6.97
146,TXN,TEXAS INSTRUMENTS INCORPORATED - COMMON STOCK ST,Purchase,1001 - 15000,Jared Moskowitz,House / D,2024-12-09,2024-11-08,-12.86
147,WST,"WEST PHARMACEUTICAL SERVICES, INC. COMMON STOC...",Sale,1001 - 15000,Jared Moskowitz,House / D,2024-12-09,2024-11-08,3.59
148,XLE,THE ENERGY SELECT SECTOR SPDR FUND OT,Purchase,15001 - 50000,Jared Moskowitz,House / D,2024-12-09,2024-11-08,-5.29
149,XOM,EXXON MOBIL CORPORATION COMMON STOCK ST,Sale,1001 - 15000,Jared Moskowitz,House / D,2024-12-09,2024-11-08,-9.12


### Loading Historical Political trades

## Need to adjust for survivorship bias eventually

In [124]:
import os

data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/Data"

political_data = pd.read_csv(os.path.join(data_folder, 'politician_trading_data_2020.csv'))

political_data['Transaction Date'] = pd.to_datetime(political_data['transaction_date'], errors='coerce')

political_data = political_data.set_index('Transaction Date')

political_data = political_data.sort_index()

political_data = political_data[(political_data.index >= '2019-01-01') & (political_data.index <= '2023-05-03')]

sp500_tickers = pd.read_csv('https://datahub.io/core/s-and-p-500-companies/r/constituents.csv')['Symbol'].tolist()

political_data = political_data[political_data['ticker'].isin(sp500_tickers)]

print(f"Filtered political data:\n\n{political_data.head()}")

Filtered political data:

                  disclosure_year disclosure_date transaction_date owner  \
Transaction Date                                                           
2019-01-09                   2020      04/27/2020       2019-01-09    --   
2019-01-09                   2020      04/27/2020       2019-01-09    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   

                 ticker             asset_description          type  \
Transaction Date                                                      
2019-01-09         DECK   Deckers Outdoor Corporation  sale_partial   
2019-01-09           RL      Ralph Lauren Corporation  sale_partial   
2019-01-23          AMD  Advanced Micro Devices, Inc.      purchase   
2019-01-23         DECK   Deckers Outdoor Corporation  sale_partial   
2019-01-23     

In [125]:
political_data.head(5)

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,state,ptr_link,cap_gains_over_200_usd,industry,sector,party
Transaction Date,,,,,,,,,,,,,,,,
2019-01-09,2020,04/27/2020,2019-01-09,--,DECK,Deckers Outdoor Corporation,sale_partial,"$1,001 - $15,000",Donna Shalala,FL27,FL,https://disclosures-clerk.house.gov/public_dis...,False,Shoe Manufacturing,Consumer Non-Durables,Democrat
2019-01-09,2020,04/27/2020,2019-01-09,--,RL,Ralph Lauren Corporation,sale_partial,"$1,001 - $15,000",Donna Shalala,FL27,FL,https://disclosures-clerk.house.gov/public_dis...,False,Apparel,Consumer Non-Durables,Democrat
2019-01-23,2020,04/27/2020,2019-01-23,--,AMD,"Advanced Micro Devices, Inc.",purchase,"$1,001 - $15,000",Donna Shalala,FL27,FL,https://disclosures-clerk.house.gov/public_dis...,False,Semiconductors,Technology,Democrat
2019-01-23,2020,04/27/2020,2019-01-23,--,DECK,Deckers Outdoor Corporation,sale_partial,"$1,001 - $15,000",Donna Shalala,FL27,FL,https://disclosures-clerk.house.gov/public_dis...,False,Shoe Manufacturing,Consumer Non-Durables,Democrat
2019-01-23,2020,04/27/2020,2019-01-23,--,PNR,Pentair plc. Ordinary Share,purchase,"$1,001 - $15,000",Donna Shalala,FL27,FL,https://disclosures-clerk.house.gov/public_dis...,False,Fluid Controls,Industrials,Democrat


### Fetch historical data to determine what the adj closing price was on the transaction date

In [119]:
reshaped_data

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
A      2019-01-09   68.199997   69.660004   68.000000   69.250000   66.356201   
       2019-01-10   69.050003   69.949997   68.599998   69.900002   66.979034   
       2019-01-11   69.290001   70.410004   68.940002   70.379997   67.438957   
       2019-01-14   69.720001   70.290001   69.669998   69.750000   66.835297   
       2019-01-15   69.949997   70.750000   69.620003   70.599998   67.649750   
...                       ...         ...         ...         ...         ...   
ZTS    2023-04-26  172.369995  173.479996  171.500000  172.940002  170.460083   
       2023-04-27  173.869995  174.130005  171.160004  173.949997  171.455582   
       2023-04-28  174.229996  177.039993  174.059998  175.779999  173.259338   
       2023-05-01  175.720001  179.940002  175.539993  179.559998  176.985153   
       2023-05-02  179.479996  179.889999  176.100006  176.899994  174.363281   

Price                 Volume  
Ticker Date                   
A      2019-01-09  2442300.0  
       2019-01-10  1080900.0  
       2019-01-11  1210800.0  
       2019-01-14  2182700.0  
       2019-01-15  1793100.0  
...                      ...  
ZTS    2023-04-26  1205200.0  
       2023-04-27  1682900.0  
       2023-04-28  1606700.0  
       2023-05-01  1689900.0  
       2023-05-02  2620300.0  

[546258 rows x 6 columns]

In [126]:
import yfinance as yf

# sp500_data = yf.download(sp500_tickers,start="2019-01-09",end="2023-05-03", group_by='ticker')

# reshaped_data = sp500_data.stack(level=0, future_stack=True).swaplevel().sort_index()

filtered_stock_data = reshaped_data.loc[political_data['ticker'].unique()]

filtered_stock_data = filtered_stock_data['Adj Close']
filtered_stock_data.head()

filtered_stock_data = filtered_stock_data.reset_index()
filtered_stock_data.head()

political_data = political_data.reset_index()
political_data = political_data[["Transaction Date", "ticker", "type", "amount", "representative"]]
political_data = political_data.rename(columns={"Transaction Date": "Date", "ticker": "Ticker", "type": "Type", "amount": "Amount", "representative": "Representative"})
political_data.head()

# Merge the two datasets
merged_data = pd.merge(
    political_data,
    filtered_stock_data,
    on=['Ticker', 'Date'],
    how="inner"
)

merged_data = merged_data.set_index(["Ticker", "Date"])

merged_data.head()


KeyError: "['index'] not found in axis"

In [129]:
merged_data['Amount'].unique()

array(['$1,001 - $15,000', '$15,001 - $50,000', '$50,001 - $100,000',
       '$100,001 - $250,000', '$1,001 -', '$1,000,001 - $5,000,000',
       '$250,001 - $500,000', '$500,001 - $1,000,000',
       '$5,000,001 - $25,000,000', '$15,000 - $50,000',
       '$1,000 - $15,000'], dtype=object)

In [130]:
# Define the mapping for amount ranges to midpoint values
amount_mapping = {
    '$1,001 - $15,000': 8000.0,
    '$15,001 - $50,000': 32500.0,
    '$50,001 - $100,000': 75000.0,
    '$100,001 - $250,000': 175000.0,
    '$1,001 -': 1001.0,  # Assume the minimum for open-ended range
    '$1,000,001 - $5,000,000': 3000000.0,
    '$250,001 - $500,000': 375000.0,
    '$500,001 - $1,000,000': 750000.0,
    '$5,000,001 - $25,000,000': 15000000.0,
    '$15,000 - $50,000': 32500.0,  # Align with similar range
    '$1,000 - $15,000': 8000.0     # Align with similar range
}

# Map the amount ranges to their midpoint values
merged_data['Amount'] = merged_data['Amount'].map(amount_mapping)

# Check the updated data
print(merged_data['Amount'].head(30))
print(merged_data['Amount'].isna().sum())

Ticker  Date      
DVN     2023-03-31     8000.0
MSFT    2023-03-31     8000.0
DVN     2023-03-31     8000.0
MRNA    2023-03-31     8000.0
KO      2023-04-04     8000.0
TXN     2023-04-05     8000.0
CI      2023-04-05     8000.0
PSX     2023-04-05    32500.0
AMGN    2023-04-05    32500.0
CTVA    2023-04-10    32500.0
MSFT    2023-04-10    32500.0
BX      2023-04-10    32500.0
GS      2023-04-10     8000.0
MSFT    2023-04-10     8000.0
GRMN    2023-04-10     8000.0
TROW    2023-04-10     8000.0
MMM     2023-04-10     8000.0
SPG     2023-04-13    32500.0
SHW     2023-04-14     8000.0
ROP     2023-04-14     8000.0
PEG     2023-04-14     8000.0
NTAP    2023-04-14     8000.0
        2023-04-14     8000.0
ADI     2023-04-14     8000.0
NVDA    2023-04-14    32500.0
KMB     2023-04-14     8000.0
IQV     2023-04-20     8000.0
AJG     2023-04-20    32500.0
CSCO    2023-04-20     8000.0
XOM     2023-04-20     8000.0
Name: Amount, dtype: float64


In [139]:
merged_data['Type'].unique()

array(['sale_partial', 'purchase', 'sale_full', 'exchange', 'sale'],
      dtype=object)

In [156]:
# Add a column for Quarter
merged_data['Quater'] = merged_data.index.get_level_values('Date').to_period('Q')

def categorise_transaction_type(transaction_type):
    transaction_type = transaction_type.lower()
    if 'purchase' in transaction_type:
        return 'buy'
    elif'sale' in transaction_type:
        return 'sell'

merged_data['Transaction Category'] = merged_data['Type'].apply(categorise_transaction_type)

# Group by Ticker and Quarter
quarterly_aggregated = merged_data.groupby(['Ticker', 'Quater']).agg(
    Total_Transactions=('Amount', 'count'),
    Total_Buy_Amount=(
        'Amount',
        lambda x: sum(
            amt for amt, trans_type in zip(x, merged_data.loc[x.index, 'Transaction Category']) if trans_type == 'buy'
        )
    ),
    Total_Sell_Amount=(
        'Amount',
        lambda x: sum(
            amt for amt, trans_type in zip(x, merged_data.loc[x.index, 'Transaction Category']) if trans_type == 'sell'
        )
    )
).reset_index()

# Calculate Net Buy-Sell Amount
quarterly_aggregated['Net_Buy_Sell'] = (
    quarterly_aggregated['Total_Buy_Amount'] - quarterly_aggregated['Total_Sell_Amount']
)

# Display the aggregated data
print(quarterly_aggregated.head())

  Ticker  Quater  Total_Transactions  Total_Buy_Amount  Total_Sell_Amount  \
0      A  2020Q2                   1               0.0             8000.0   
1      A  2021Q1                   1               0.0             8000.0   
2   AAPL  2019Q1                   1               0.0             8000.0   
3   AAPL  2019Q3                   3               0.0            48500.0   
4   AAPL  2019Q4                  12           48500.0           145500.0   

   Net_Buy_Sell  
0       -8000.0  
1       -8000.0  
2       -8000.0  
3      -48500.0  
4      -97000.0  


In [160]:
quarterly_aggregated = quarterly_aggregated[quarterly_aggregated['Total_Transactions']>=5]
quarterly_aggregated['Signal'] = quarterly_aggregated.apply(
    lambda row: 'Buy' if row['Total_Buy_Amount'] > 2 * row['Total_Sell_Amount'] 
    else ('Sell' if row['Total_Sell_Amount'] > 2 * row['Total_Buy_Amount'] else 'Hold'), 
    axis=1
)

In [162]:
quarterly_aggregated.head()

,Ticker,Quater,Total_Transactions,Total_Buy_Amount,Total_Sell_Amount,Net_Buy_Sell,Signal
4,AAPL,2019Q4,12,48500.0,145500.0,-97000.0,Sell
5,AAPL,2020Q1,32,1011000.0,7113501.0,-6102501.0,Sell
6,AAPL,2020Q2,25,4679500.0,3129000.0,1550500.0,Hold
7,AAPL,2020Q3,21,41501.0,315500.0,-273999.0,Sell
8,AAPL,2020Q4,18,170000.0,1254500.0,-1084500.0,Sell
